In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import os

In [2]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split  # EmoBank의 'split' 컬럼을 활용하지만, 일반적인 분할 예시도 보여주기 위해 남겨둡니다.

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoModel, BertModel, get_linear_schedule_with_warmup # AutoModel은 다양한 트랜스포머 모델을 유연하게 로드

from tqdm.notebook import tqdm

from scipy.stats import pearsonr

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
BASE_DIR = "/content/drive/MyDrive/Ajou_ISE/4학년_1학기/B113-1 딥러닝응용/팀프로젝트 1조/dataset"

In [5]:
emobank_file_name = 'main-dataset-emobank.csv'
emobank_path = os.path.join(BASE_DIR, emobank_file_name)

In [6]:
eb = pd.read_csv(emobank_path, index_col=0, encoding='latin1')

In [7]:
print(eb.head())
print(f"\n데이터셋 전체 크기: {eb.shape}")

                              split     V     A     D  \
id                                                      
vampires_4446_4474            train  4.60  4.30  3.70   
20020731-nyt_25143_25174      train  4.56  4.00  3.78   
captured_moments_33365_33387  train  4.40  4.10  3.80   
captured_moments_28753_28863  train  4.38  3.88  3.13   
captured_moments_7553_7566    train  4.37  4.12  3.50   

                                                                           text  \
id                                                                                
vampires_4446_4474                                 lol Wonderful Simply Superb!   
20020731-nyt_25143_25174                        "I am thrilled with the price."   
captured_moments_33365_33387                             "Tell her I love her."   
captured_moments_28753_28863  For a perfect moment, Emil and Tasha and I wer...   
captured_moments_7553_7566                                        I'm in love."   

                  

In [8]:
print(eb.head(20))

                                       split     V     A     D  \
id                                                               
vampires_4446_4474                     train  4.60  4.30  3.70   
20020731-nyt_25143_25174               train  4.56  4.00  3.78   
captured_moments_33365_33387           train  4.40  4.10  3.80   
captured_moments_28753_28863           train  4.38  3.88  3.13   
captured_moments_7553_7566             train  4.37  4.12  3.50   
littleshelter2_1448_1478               train  4.37  4.37  3.18   
SemEval_759                            train  4.33  4.22  3.67   
112C-L015_1820_1880                    train  4.30  3.60  2.90   
Acephalous-Internet_1795_1799          train  4.30  4.30  3.10   
captured_moments_28728_28752           train  4.30  4.40  3.40   
detroit_11417_11443                    train  4.30  3.60  3.10   
114CUL058_1185_1212                    train  4.20  4.00  3.30   
NYTnewswire1_3167_3223                 train  4.20  4.20  3.20   
Seedbombin

In [9]:
df = eb[['text', 'V', 'A', 'D', 'split']].copy()

In [10]:
print(f"결측값 제거 전 크기: {df.shape}")

결측값 제거 전 크기: (9436, 5)


In [11]:
print(df.isnull().sum())
initial_rows = df.shape[0]
df.dropna(inplace=True)
rows_after_dropna = df.shape[0]

text     1
V        0
A        0
D        0
split    0
dtype: int64


In [12]:
print(f"결측값 제거 후 크기: {df.shape}")

결측값 제거 후 크기: (9435, 5)


In [13]:
df.head()

,text,V,A,D,split
id,,,,,
vampires_4446_4474,lol Wonderful Simply Superb!,4.60,4.30,3.70,train
20020731-nyt_25143_25174,"""I am thrilled with the price.""",4.56,4.00,3.78,train
captured_moments_33365_33387,"""Tell her I love her.""",4.40,4.10,3.80,train
captured_moments_28753_28863,"For a perfect moment, Emil and Tasha and I wer...",4.38,3.88,3.13,train
captured_moments_7553_7566,"I'm in love.""",4.37,4.12,3.50,train


In [14]:
df = df.reset_index()
df = df[['text', 'V', 'A', 'D', 'split']].copy()

In [15]:
df.head()

,text,V,A,D,split
0,lol Wonderful Simply Superb!,4.60,4.30,3.70,train
1,"""I am thrilled with the price.""",4.56,4.00,3.78,train
2,"""Tell her I love her.""",4.40,4.10,3.80,train
3,"For a perfect moment, Emil and Tasha and I wer...",4.38,3.88,3.13,train
4,"I'm in love.""",4.37,4.12,3.50,train


In [16]:
# model_name = "bert-base-uncased"  # bert는 최대 길이가 512라서 1024 시퀀스 처리 불가
model_name = "allenai/longformer-base-4096"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [17]:
MAX_SEQUENCE_LENGTH = 1024

In [18]:
encoded_data = tokenizer.batch_encode_plus(
        df['text'].tolist(), # 'text' 컬럼의 모든 텍스트를 리스트로 변환
        add_special_tokens=True, # [CLS], [SEP] 같은 특수 토큰 추가
        max_length=MAX_SEQUENCE_LENGTH,          # 최대 토큰 길이 (최대가 834)
        padding='max_length',    # 모든 시퀀스를 동일한 길이로 패딩
        truncation=True,         # 최대 길이를 초과하는 시퀀스 자르기
        return_attention_mask=True, # 어텐션 마스크 반환
        return_tensors='pt'      # PyTorch 텐서로 반환
    )

기존 Dataframe의 'text' column 중 가장 긴 길이

In [19]:
max_len = df['text'].str.len().max()
print("최대 길이:", max_len)

최대 길이: 834


In [20]:
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']

print(f"\n인코딩된 input_ids의 형태: {input_ids.shape}")
print(f"인코딩된 attention_mask의 형태: {attention_mask.shape}")
print(f"첫 번째 텍스트의 input_ids 예시: {input_ids[0][:10]}...")
print(f"첫 번째 텍스트의 디코딩 예시: {tokenizer.decode(input_ids[0], skip_special_tokens=True)[:50]}...")



인코딩된 input_ids의 형태: torch.Size([9435, 1024])
인코딩된 attention_mask의 형태: torch.Size([9435, 1024])
첫 번째 텍스트의 input_ids 예시: tensor([    0, 46078, 38022, 17449,  1582,   428,   328,     2,     1,     1])...
첫 번째 텍스트의 디코딩 예시: lol Wonderful Simply Superb!...


In [21]:
print("감성 레이블(VAD)을 PyTorch 텐서로 변환")
labels = torch.tensor(df[['V', 'A', 'D']].values, dtype=torch.float32)

print(f"레이블 텐서의 형태: {labels.shape}")

감성 레이블(VAD)을 PyTorch 텐서로 변환
레이블 텐서의 형태: torch.Size([9435, 3])


In [22]:
train_indices = df[df['split'] == 'train'].index
test_indices = df[df['split'] == 'test'].index
dev_indices = df[df['split'] == 'dev'].index

In [23]:
train_input_ids = input_ids[train_indices]
train_attention_mask = attention_mask[train_indices]
train_labels = labels[train_indices]

In [24]:
test_input_ids = input_ids[test_indices]
test_attention_mask = attention_mask[test_indices]
test_labels = labels[test_indices]

In [25]:
dev_input_ids = input_ids[dev_indices]
dev_attention_mask = attention_mask[dev_indices]
dev_labels = labels[dev_indices]

In [26]:
print(f"\n학습(Train) 세트 크기: {len(train_labels)}개")
print(f"검증(Dev) 세트 크기: {len(dev_labels)}개")
print(f"테스트(Test) 세트 크기: {len(test_labels)}개")


학습(Train) 세트 크기: 7568개
검증(Dev) 세트 크기: 940개
테스트(Test) 세트 크기: 927개


In [27]:
model_name = "allenai/longformer-base-4096"

In [28]:
class EmotionRegressor(nn.Module):
    def __init__(self, model_name):
        super(EmotionRegressor, self).__init__()
        # 3-1. 사전 학습된 Transformer 모델 로드
        # AutoModel.from_pretrained()를 사용하여 지정된 model_name에 맞는 모델을 로드합니다.
        # 이 모델은 텍스트의 context-aware 임베딩을 생성합니다.
        print(f"'{model_name}' 사전 학습 모델을 로드")
        self.bert = AutoModel.from_pretrained(model_name)

        # 3-2. 회귀를 위한 Linear 레이어 추가 (출력 헤드)
        # BERT 모델의 마지막 히든 레이어 크기(보통 768)를 입력으로 받아,
        # V, A, D 세 가지 감성 값을 예측하기 위해 출력 차원을 3으로 설정합니다.
        # EmoBank의 VAD 스케일이 1-5이므로, 출력 범위에 대한 활성화 함수는 나중에 고려할 수 있습니다.
        self.regressor = nn.Linear(self.bert.config.hidden_size, 3)

    def forward(self, input_ids, attention_mask):
        # 3-3. Transformer 모델에 입력 전달
        # input_ids: 토큰 ID 시퀀스
        # attention_mask: 패딩 토큰을 무시하기 위한 마스크
        # outputs: Transformer 모델의 출력을 담는 객체
        # outputs.pooler_output: [CLS] 토큰에 해당하는 최종 히든 스테이트 (일반적으로 전체 문장의 요약 임베딩으로 사용)
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # [CLS] 토큰의 임베딩을 문장 전체의 감성을 대표하는 벡터로 사용합니다.
        cls_embedding = outputs.pooler_output

        # 3-4. Linear 레이어를 통해 VAD 값 예측
        predictions = self.regressor(cls_embedding)
        return predictions

In [29]:
model = EmotionRegressor(model_name)

'allenai/longformer-base-4096' 사전 학습 모델을 로드


pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

EmotionRegressor(
  (bert): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linea

### Model Params 확인

In [31]:
!pip install torchinfo

from torchinfo import summary

model.safetensors:   0%|          | 0.00/597M [00:00<?, ?B/s]

In [32]:
total_params = sum(p.numel() for p in model.parameters())
trainable   = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total: {total_params:,}개, Trainable: {trainable:,}개")

Total: 148,661,763개, Trainable: 148,661,763개


### DataLoader

In [33]:
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=8)
print(f"학습(Train) 데이터 로더 생성 완료. 샘플 수: {len(train_dataset)}")


학습(Train) 데이터 로더 생성 완료. 샘플 수: 7568


In [34]:
dev_dataset = TensorDataset(dev_input_ids, dev_attention_mask, dev_labels)
dev_sampler = SequentialSampler(dev_dataset)
dev_dataloader = DataLoader(dev_dataset, sampler=dev_sampler, batch_size=8)
print(f"검증(Dev) 데이터 로더 생성 완료. 샘플 수: {len(dev_dataset)}")


검증(Dev) 데이터 로더 생성 완료. 샘플 수: 940


In [35]:
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=8)
print(f"테스트(Test) 데이터 로더 생성 완료. 샘플 수: {len(test_dataset)}")

테스트(Test) 데이터 로더 생성 완료. 샘플 수: 927


In [36]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [37]:
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
print(f"총 학습 에포크: {epochs}, 총 학습 스텝: {total_steps}")

총 학습 에포크: 5, 총 학습 스텝: 4730


In [38]:
criterion = nn.MSELoss()

In [39]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [40]:
for epoch_i in range(0, epochs):
    print(f"\n======== Epoch {epoch_i + 1} / {epochs} ========")
    print("Training...")

    model.train()
    total_train_loss = 0

    for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, b_attention_mask)

        loss = criterion(outputs, b_labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"평균 학습 손실: {avg_train_loss:.4f}")

    # 4-5. 검증 (Validation) 단계 (dev_dataloader가 있는 경우만 실행)
    if dev_dataloader is not None:
        print("\n검증(Validation)을 수행합니다...")
        model.eval()
        total_eval_loss = 0
        predictions_list = []
        true_labels_list = []

        for batch in tqdm(dev_dataloader, desc="Validating"):
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            with torch.no_grad():
                outputs = model(b_input_ids, b_attention_mask)

            loss = criterion(outputs, b_labels)
            total_eval_loss += loss.item()

            predictions_list.append(outputs.cpu().numpy())
            true_labels_list.append(b_labels.cpu().numpy())

        avg_eval_loss = total_eval_loss / len(dev_dataloader)
        print(f"검증 손실: {avg_eval_loss:.4f}")

        # 평가 지표 계산 (Pearson Correlation, RMSE)
        predictions = np.concatenate(predictions_list, axis=0)
        true_labels = np.concatenate(true_labels_list, axis=0)

        v_corr = pearsonr(predictions[:, 0], true_labels[:, 0])[0]
        a_corr = pearsonr(predictions[:, 1], true_labels[:, 1])[0]
        d_corr = pearsonr(predictions[:, 2], true_labels[:, 2])[0]

        v_rmse = np.sqrt(np.mean((predictions[:, 0] - true_labels[:, 0])**2))
        a_rmse = np.sqrt(np.mean((predictions[:, 1] - true_labels[:, 1])**2))
        d_rmse = np.sqrt(np.mean((predictions[:, 2] - true_labels[:, 2])**2))

        print(f"검증 V_Corr: {v_corr:.4f}, A_Corr: {a_corr:.4f}, D_Corr: {d_corr:.4f}")
        print(f"검증 V_RMSE: {v_rmse:.4f}, A_RMSE: {a_rmse:.4f}, D_RMSE: {d_rmse:.4f}")



======== Epoch 1 / 5 ========
Training...


Training:   0%|          | 0/946 [00:00<?, ?it/s]

평균 학습 손실: 0.1884

검증(Validation)을 수행합니다...


Validating:   0%|          | 0/118 [00:00<?, ?it/s]

검증 손실: 0.0521
검증 V_Corr: 0.7542, A_Corr: 0.5373, D_Corr: 0.3741
검증 V_RMSE: 0.2412, A_RMSE: 0.2287, D_RMSE: 0.2085

======== Epoch 2 / 5 ========
Training...


...

검증 손실: 0.0472
검증 V_Corr: 0.8037, A_Corr: 0.5610, D_Corr: 0.4707
검증 V_RMSE: 0.2272, A_RMSE: 0.2220, D_RMSE: 0.1965

======== Epoch 5 / 5 ========
Training...


Training:   0%|          | 0/946 [00:00<?, ?it/s]

평균 학습 손실: 0.0219

검증(Validation)을 수행합니다...


Validating:   0%|          | 0/118 [00:00<?, ?it/s]

검증 손실: 0.0475
검증 V_Corr: 0.8066, A_Corr: 0.5682, D_Corr: 0.4777
검증 V_RMSE: 0.2263, A_RMSE: 0.2261, D_RMSE: 0.1951


In [41]:
print("\n모델 학습 완료. 최종 테스트를 수행합니다...")
model.eval()
total_test_loss = 0
predictions_list = []
true_labels_list = []

for batch in tqdm(test_dataloader, desc="Testing"):
    b_input_ids = batch[0].to(device)
    b_attention_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        outputs = model(b_input_ids, b_attention_mask)

    loss = criterion(outputs, b_labels)
    total_test_loss += loss.item()

    predictions_list.append(outputs.cpu().numpy())
    true_labels_list.append(b_labels.cpu().numpy())

avg_test_loss = total_test_loss / len(test_dataloader)
print(f"최종 테스트 손실: {avg_test_loss:.4f}")

predictions = np.concatenate(predictions_list, axis=0)
true_labels = np.concatenate(true_labels_list, axis=0)

v_corr = pearsonr(predictions[:, 0], true_labels[:, 0])[0]
a_corr = pearsonr(predictions[:, 1], true_labels[:, 1])[0]
d_corr = pearsonr(predictions[:, 2], true_labels[:, 2])[0]

v_rmse = np.sqrt(np.mean((predictions[:, 0] - true_labels[:, 0])**2))
a_rmse = np.sqrt(np.mean((predictions[:, 1] - true_labels[:, 1])**2))
d_rmse = np.sqrt(np.mean((predictions[:, 2] - true_labels[:, 2])**2))

print(f"최종 테스트 V_Corr: {v_corr:.4f}, A_Corr: {a_corr:.4f}, D_Corr: {d_corr:.4f}")
print(f"최종 테스트 V_RMSE: {v_rmse:.4f}, A_RMSE: {a_rmse:.4f}, D_RMSE: {d_rmse:.4f}")

# 학습된 모델 저장 (선택 사항)
# from datetime import datetime
# current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
# model_save_path = os.path.join(base_dir, f'emobank_emotion_regressor_{current_time}.pth')
# torch.save(model.state_dict(), model_save_path)
# print(f"학습된 모델이 다음 경로에 저장되었습니다: {model_save_path}")

print("\n[단계 4: 데이터 로더 생성 및 모델 학습/평가 완료]")


모델 학습 완료. 최종 테스트를 수행합니다...


Testing:   0%|          | 0/116 [00:00<?, ?it/s]

최종 테스트 손실: 0.0468
최종 테스트 V_Corr: 0.8118, A_Corr: 0.5310, D_Corr: 0.5258
최종 테스트 V_RMSE: 0.2252, A_RMSE: 0.2239, D_RMSE: 0.1972

[단계 4: 데이터 로더 생성 및 모델 학습/평가 완료]
